# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install langchain_openai langchain_huggingface langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 44.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7


### Example 1

#### Create our QandA application

In [5]:
# from langchain.chains import RetrievalQA
# from langchain_openai import ChatOpenAI
# from langchain.llms import OpenAI
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.document_loaders import CSVLoader, TextLoader
# from langchain.indexes import VectorstoreIndexCreator
# from langchain.vectorstores import DocArrayInMemorySearch
# from langchain.chains import LLMChain

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents import create_agent
from langchain_classic.tools.retriever import create_retriever_tool
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [6]:
from langchain_community.document_loaders import CSVLoader, TextLoader

In [7]:
file = '/content/data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
doc_splits = text_splitter.split_documents(data)

# Create embeddings with HuggingFace
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Create vector store and add documents
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}  # Return top 5 most similar documents
)

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create a simple RAG chain
template = """Use the following context to answer the question:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


llm = ChatOpenAI(temperature = 0.0,api_key="your api key")
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=index.vectorstore.as_retriever(),
#     verbose=True,
#     chain_type_kwargs = {
#         "document_separator": "<<<<>>>>>"
#     }
# )
chain = (
    {
        "context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
result = chain.invoke("Do the Cozy Comfort Pullover Set")
print(result)

and the Cozy Cuddles Knit Pullover Set have the same fabric composition?


#### Coming up with test datapoints

In [11]:
data[10]

Document(metadata={'source': '/content/data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [12]:
data[11]

Document(metadata={'source': '/content/data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [13]:
from langchain_core.prompts import PromptTemplate


In [14]:
from langchain_core.output_parsers import BaseOutputParser

from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    """Examples:
1. Query: Do the Cozy Comfort Pullover Set have side pockets?
   Answer: Yes
2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?
   Answer: The DownTek collection

Query: {query}
Answer:"""
)
# prompt_template = ChatPromptTemplate.from_template(
#     "Question: {question}\nAnswer:"
# )

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
# llm = ChatOpenAI()

# Create the LLMChain
# llm_chain = LLMChain(
#     llm=llm,
#     prompt=prompt_template,
#     output_parser=AnswerOutputParser()
# )
llm_chain=prompt | llm | AnswerOutputParser()
# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
# result = llm_chain.invoke({"query": query})
result = llm_chain.invoke({
    "question": query,       # matches {question}
    "context": ""            # or actual context string that your prompt expects
})

# Print the result
print(result)


answer='Yes, the Cozy Comfort Pullover Set is available in multiple colors.'


#### LLM-Generated examples

In [15]:
from langchain_classic.evaluation.qa import QAGenerateChain

In [16]:
example_gen_chain = QAGenerateChain.from_llm(llm)

In [17]:
llm_chain_qa = prompt_template | llm

In [18]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/tmp/ipython-input-3125874183.py:1: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_examples = example_gen_chain.apply_and_parse(


In [19]:
new_examples[0]

{'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."}}

In [20]:
data[0]

Document(metadata={'source': '/content/data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [21]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "Some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom for secure fit and maximum coverage, and the recommendation to machine wash and line dry for best results."}

#### Combine examples

In [22]:
# examples += new_example
examples += d_flattened

In [23]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [24]:
llm_chain_qa.invoke(examples[0]["query"])

AIMessage(content='Yes, the Cozy Comfort Pullover Set does have side pockets.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 76, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CfqUNUTDnwtwzY2MxW5nAKyhY3Ob3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ec5b1295-9e4f-4401-aec4-56d2256968a5-0', usage_metadata={'input_tokens': 76, 'output_tokens': 14, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Manual Evaluation - Fun part

In [25]:
import langchain
langchain.debug = True

In [26]:
llm_chain_qa.invoke(examples[0]["query"])

AIMessage(content='Yes, the Cozy Comfort Pullover Set does have side pockets.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 76, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CfqUOFcPum3i6yHCjhsbmyxkAFxfm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4d98bcbf-58b3-49fc-a843-28da7e4e442a-0', usage_metadata={'input_tokens': 76, 'output_tokens': 14, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [28]:
examples += d_flattened

In [29]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "Some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch

In [30]:
predictions = llm_chain_qa.batch(examples)

In [31]:
predictions

[AIMessage(content='Yes, the Cozy Comfort Pullover Set does have side pockets.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 76, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CfqUPNX4AR6wP6H413JCzrHSEfmjP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d473161a-ed64-4df0-aaf0-92020c930b56-0', usage_metadata={'input_tokens': 76, 'output_tokens': 14, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='The DownTek collection', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'comple

In [32]:
from langchain_classic.evaluation.qa import QAEvalChain

In [33]:
llm = ChatOpenAI(temperature=0,api_key="your api key")
eval_chain = QAEvalChain.from_llm(llm)

In [34]:
print(type(predictions[0]))
print(predictions[0])

<class 'langchain_core.messages.ai.AIMessage'>
content='Yes, the Cozy Comfort Pullover Set does have side pockets.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 76, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CfqUPNX4AR6wP6H413JCzrHSEfmjP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--d473161a-ed64-4df0-aaf0-92020c930b56-0' usage_metadata={'input_tokens': 76, 'output_tokens': 14, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [35]:
predictions_fixed = [{"prediction": p.content} for p in predictions]

In [36]:
print("example[0]:", examples[0])
print("prediction[0]:", predictions_fixed[0])

example[0]: {'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}
prediction[0]: {'prediction': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}


In [37]:
# graded_outputs = eval_chain.evaluate(examples, predictions)
graded_outputs = eval_chain.evaluate(
    examples,
    predictions_fixed,
    question_key="query",    # key name in your examples
    answer_key="answer",        # key name in your examples
    prediction_key="prediction" # key used in predictions_fixed
)

In [39]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [38]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + examples[i]['query'])
    print("Real Answer: " + examples[i]['answer'])
    print("Predicted Answer: " + predictions_fixed[i]['prediction'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])


Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The DownTek collection
Example 2:
Question: What is the approximate weight of the Women's Campside Oxfords per pair?
Real Answer: The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz.
Predicted Answer: The approximate weight of the Women's Campside Oxfords per pair is 1 lb 6 oz.
Example 3:
Question: What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".
Predicted Answer: The small size is 18" x 27" and the medium size is 24" x 36".
Ex

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [52]:
pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.2 MB/s eta 0:00:00


In [53]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
loader = TextLoader("/content/data/nyc_text.txt")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs = loader.load()
doc_splits = text_splitter.split_documents(docs)

# Create embeddings with HuggingFace
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Create vector store and add documents
vectorstore = DocArrayInMemorySearch.from_documents(doc_splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

llm = llm
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=index.vectorstore.as_retriever(),
#     return_source_documents=True,
# )
qa_chain = (
    {
        "context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
        "question": RunnablePassthrough()
    }
    |prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.12/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [54]:
def get_docs_for_question(retriever, question):
    if hasattr(retriever, "get_relevant_documents"):
        return retriever.get_relevant_documents(question)
    if hasattr(retriever, "invoke"):
        return retriever.invoke(question)
    # fallback: assume callable
    return retriever(question)

In [55]:
def rag_answer(question: str) -> str:
    docs = get_docs_for_question(retriever, question)
    def doc_text(d):
        if hasattr(d, "page_content"):
            return d.page_content
        if isinstance(d, dict) and "page_content" in d:
            return d["page_content"]
        return str(d)

    context = "\n\n".join(doc_text(d) for d in docs)
    result = qa_chain.invoke({"question": question, "context": context})
    return result

In [57]:
# testing it out

question = "How did New York City get its name?"
answer = qa_chain.invoke(question)
print("Answer:", answer)

Answer: New York City was originally named New Amsterdam by Dutch colonists in 1626. It was later renamed New York after King Charles II of England granted the lands to his brother, the Duke of York, when the city came under British control in 1664.


In [58]:
result

Answer(answer='Yes, the Cozy Comfort Pullover Set is available in multiple colors.')

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [59]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [60]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [63]:
question=eval_questions[1]

answer = qa_chain.invoke(question)
print("Answer:", answer)

Answer: Manhattan (New York County) has the highest population density of any borough in New York City.


In [64]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [65]:
result_updated

{}

In [67]:
# !pip install --no-cache-dir recordclass

In [66]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 155.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 46.1 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.12.0
    Uninstalling jiter-0.12.0:
      Successfully uninstalled jiter-0.12.0


In [1]:
from ragas.integrations.langchain import EvaluatorChain
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

ModuleNotFoundError: No module named 'langchain.chains'

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [ ]:
# Recheck the result that we are going to validate.
result

**Faithfulness**

In [ ]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness_score"]

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [ ]:
fake_result = result.copy()
fake_result["result"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness_score"]

**Context Relevancy**

In [ ]:
eval_result = context_recall_chain(result)
eval_result["context_recall_score"]

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [ ]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [ ]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

In [ ]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r